# Яндекс Недвижимость

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
import re
import random
import math
import numpy as np

In [2]:
pd.__version__

'1.4.2'

In [3]:
# set max columns to none
pd.set_option("display.max_columns", None)

if type of location not set assumming that its town

In [4]:
def name_n_type(text):
        whole_words = text.split()
        type_list = []
        name_list = []
        name_loc = " "
        type_loc = " "
        
        # regex
        pattern = '[А-ЯЁ]+[а-яё]'
        
        for word in whole_words:
            
            # searching pattern
            if re.search(pattern, word):
                name_list.append(word)
            else:
                type_list.append(word)
        
        name_loc = name_loc.join(name_list)
        
        if len(type_list) == 0:
            type_loc = 'город'
        else:
            type_loc = type_loc.join(type_list)
        
        return name_loc, type_loc

In [5]:
def correct_name(to_correct):
    # print(to_correct)
    if 'поселок' in to_correct:
        to_correct = to_correct.replace('поселок', 'посёлок')
        # print(to_correct)
    return to_correct

In [6]:
# test_list = [
#  'деревня Новое Девяткино',
#  'посёлок Металлострой',
#  'посёлок городского типа Лебяжье',
#  'посёлок городского типа Сиверский',
#  'посёлок Молодцово',
#  'посёлок городского типа Кузьмоловский',
#  'садовое товарищество Новая Ропша',
#  'Павловск',
#  'деревня Пикколово',
#  'Всеволожск',
#  'Волхов',
#  'Кингисепп',
#  'Приозерск',
#  'Сестрорецк',
#  'деревня Куттузи',
#  'посёлок Аннино',
#  'посёлок городского типа Ефимовский',
# ]

# for word in test_list:
#     print(name_n_type(word))

In [7]:
real_estate_data = pd.read_csv('./datasets/real_estate_data.csv', sep='\t')

In [8]:
display(real_estate_data['balcony'].unique())

array([nan,  2.,  0.,  1.,  5.,  4.,  3.])

In [9]:
display(real_estate_data.head())

total_images  last_price  total_area first_day_exposition  rooms  \
0            20  13000000.0       108.0  2019-03-07T00:00:00      3   
1             7   3350000.0        40.4  2018-12-04T00:00:00      1   
2            10   5196000.0        56.0  2015-08-20T00:00:00      2   
3             0  64900000.0       159.0  2015-07-24T00:00:00      3   
4             2  10000000.0       100.0  2018-06-19T00:00:00      2   

   ceiling_height  floors_total  living_area  floor is_apartment  studio  \
0            2.70          16.0         51.0      8          NaN   False   
1             NaN          11.0         18.6      1          NaN   False   
2             NaN           5.0         34.3      4          NaN   False   
3             NaN          14.0          NaN      9          NaN   False   
4            3.03          14.0         32.0     13          NaN   False   

   open_plan  kitchen_area  balcony    locality_name  airports_nearest  \
0      False          25.0      NaN  Санкт-Петербург           18863.0   
1      False          11.0      2.0   посёлок Шушары           12817.0   
2      False           8.3      0.0  Санкт-Петербург           21741.0   
3      False           NaN      0.0  Санкт-Петербург           28098.0   
4      False          41.0      NaN  Санкт-Петербург           31856.0   

   cityCenters_nearest  parks_around3000  parks_nearest  ponds_around3000  \
0              16028.0               1.0          482.0               2.0   
1              18603.0               0.0            NaN               0.0   
2              13933.0               1.0           90.0               2.0   
3               6800.0               2.0           84.0               3.0   
4               8098.0               2.0          112.0               1.0   

   ponds_nearest  days_exposition  
0          755.0              NaN  
1            NaN             81.0  
2          574.0            558.0  
3          234.0            424.0  
4           48.0            121.0

`'cityCenters_nearest'` → convert to snake case

In [10]:
real_estate_data.rename(columns={'cityCenters_nearest': 'city_centers_nearest'}, inplace=True)

In [11]:
display(real_estate_data['is_apartment'].unique())

array([nan, False, True], dtype=object)

In [12]:
display(real_estate_data['balcony'].unique())

array([nan,  2.,  0.,  1.,  5.,  4.,  3.])

<div class="alert alert-info" role="alert">
    <p class="mb-0">Как получить <code>list</code> колонок с <code>NaN</code> нашёл на <code>stackoverflow</code> 👌</p>
</div>

In [13]:
display(
    real_estate_data.columns[real_estate_data.isna().any()].tolist()
)

['ceiling_height',
 'floors_total',
 'living_area',
 'is_apartment',
 'kitchen_area',
 'balcony',
 'locality_name',
 'airports_nearest',
 'city_centers_nearest',
 'parks_around3000',
 'parks_nearest',
 'ponds_around3000',
 'ponds_nearest',
 'days_exposition']

Добро, вот с этими колонками нужно разобраться в них пропуски — `NaN`

In [14]:
display(real_estate_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_images          23699 non-null  int64  
 1   last_price            23699 non-null  float64
 2   total_area            23699 non-null  float64
 3   first_day_exposition  23699 non-null  object 
 4   rooms                 23699 non-null  int64  
 5   ceiling_height        14504 non-null  float64
 6   floors_total          23613 non-null  float64
 7   living_area           21796 non-null  float64
 8   floor                 23699 non-null  int64  
 9   is_apartment          2775 non-null   object 
 10  studio                23699 non-null  bool   
 11  open_plan             23699 non-null  bool   
 12  kitchen_area          21421 non-null  float64
 13  balcony               12180 non-null  float64
 14  locality_name         23650 non-null  object 
 15  airports_nearest   

None

In [15]:

for column in real_estate_data.columns:
    count_of_nan = len(real_estate_data.loc[real_estate_data[column].isna()][column])
    if count_of_nan > 0:
        print(column, end=' NaN = ')
        print(count_of_nan)
        print(real_estate_data[column].describe(), end='\n\n')

ceiling_height NaN = 9195
count    14504.000000
mean         2.771499
std          1.261056
min          1.000000
25%          2.520000
50%          2.650000
75%          2.800000
max        100.000000
Name: ceiling_height, dtype: float64

floors_total NaN = 86
count    23613.000000
mean        10.673824
std          6.597173
min          1.000000
25%          5.000000
50%          9.000000
75%         16.000000
max         60.000000
Name: floors_total, dtype: float64

living_area NaN = 1903
count    21796.000000
mean        34.457852
std         22.030445
min          2.000000
25%         18.600000
50%         30.000000
75%         42.300000
max        409.700000
Name: living_area, dtype: float64

is_apartment NaN = 20924
count      2775
unique        2
top       False
freq       2725
Name: is_apartment, dtype: object

kitchen_area NaN = 2278
count    21421.000000
mean        10.569807
std          5.905438
min          1.300000
25%          7.000000
50%          9.100000
75%         

## Шаг 2. Предобработка данных

Определите и изучите пропущенные значения:
Для некоторых пропущенных значений можно предположить логичную замену. Например, если человек не указал число балконов — скорее всего, их нет. Такие пропуски правильно заменить на 0. Для других типов данных нет подходящего значения на замену. В этом случае правильно оставить эти значения пустыми. Отсутствие значения — тоже важный сигнал, который не нужно прятать.



Заполните пропуски, где это уместно. Опишите, почему вы решили заполнить пропуски именно в этих столбцах и как выбрали значения.
Укажите причины, которые могли привести к пропускам в данных.

In [16]:
real_estate_data['is_apartment'].fillna(value=False, inplace=True)

In [17]:
display(
    real_estate_data['floor'].loc[(real_estate_data['balcony'].isna())].describe()
)

display(
    real_estate_data['floor'].loc[~(real_estate_data['balcony'].isna())].describe()
)

display(
    real_estate_data['floor']
        .loc[(real_estate_data['balcony'].isna())
            & (real_estate_data['floor'] == 1)]
        .count()
)

count    11519.000000
mean         5.314003
std          4.667232
min          1.000000
25%          2.000000
50%          4.000000
75%          7.000000
max         31.000000
Name: floor, dtype: float64

count    12180.000000
mean         6.439327
std          5.022114
min          1.000000
25%          3.000000
50%          5.000000
75%          9.000000
max         33.000000
Name: floor, dtype: float64

1926

In [18]:
(
    real_estate_data['balcony'].loc[(real_estate_data['balcony'].isna())
                                & (real_estate_data['floor'] == 1)]
                            .fillna(value=0, inplace=True)
)

In [19]:
display(
    real_estate_data['balcony'].unique()
)

array([nan,  2.,  0.,  1.,  5.,  4.,  3.])

next step is temp

In [20]:
real_estate_data['balcony'].fillna(value=108, inplace=True)

In [21]:
display(
    real_estate_data['balcony'].unique()
)

array([108.,   2.,   0.,   1.,   5.,   4.,   3.])

In [22]:
real_estate_data['is_apartment'].fillna(value=False, inplace=True)

In [23]:
display(
    real_estate_data.loc[real_estate_data['locality_name'].isna()]
                        .head(10)
)

total_images  last_price  total_area first_day_exposition  rooms  \
1097             3   8600000.0        81.7  2016-04-15T00:00:00      3   
2033             6   5398000.0        80.0  2017-05-30T00:00:00      3   
2603            20   3351765.0        42.7  2015-09-20T00:00:00      1   
2632             2   5130593.0        62.4  2015-10-11T00:00:00      2   
3574            10   4200000.0        46.5  2016-05-28T00:00:00      2   
4151            17  17600000.0        89.5  2014-12-09T00:00:00      2   
4189             7   9200000.0        80.0  2015-12-10T00:00:00      3   
4670             1   5500000.0        83.0  2015-08-14T00:00:00      3   
5343            19  13540000.0        85.5  2016-01-20T00:00:00      3   
5707             7   3700000.0        30.0  2016-04-29T00:00:00      1   

      ceiling_height  floors_total  living_area  floor  is_apartment  studio  \
1097            3.55           5.0        50.80      2         False   False   
2033             NaN           4.0        42.60      2         False   False   
2603             NaN          24.0        15.60      3         False   False   
2632             NaN          24.0        33.10     21         False   False   
3574             NaN           5.0        30.80      5         False   False   
4151            3.00           8.0        39.62      7         False   False   
4189            4.00           4.0        52.30      3         False   False   
4670             NaN           7.0          NaN      6         False   False   
5343             NaN           7.0        59.10      5         False   False   
5707             NaN          24.0        20.00     23         False   False   

      open_plan  kitchen_area  balcony locality_name  airports_nearest  \
1097      False          8.80      0.0           NaN           23478.0   
2033      False         18.60      0.0           NaN               NaN   
2603      False         10.70      0.0           NaN           22041.0   
2632      False          8.20      0.0           NaN           22041.0   
3574      False          6.50      0.0           NaN           27419.0   
4151      False         13.38      0.0           NaN           25054.0   
4189      False         10.40      0.0           NaN           21774.0   
4670      False           NaN      0.0           NaN           26534.0   
5343      False          8.30      4.0           NaN           10556.0   
5707      False           NaN      0.0           NaN           21460.0   

      city_centers_nearest  parks_around3000  parks_nearest  ponds_around3000  \
1097                4258.0               0.0            NaN               0.0   
2033                   NaN               NaN            NaN               NaN   
2603               17369.0               0.0            NaN               1.0   
2632               17369.0               0.0            NaN               1.0   
3574                8127.0               0.0            NaN               1.0   
4151                3902.0               1.0          485.0               3.0   
4189                3039.0               1.0          690.0               1.0   
4670                5382.0               1.0          446.0               1.0   
5343                9538.0               1.0           67.0               0.0   
5707               16788.0               0.0            NaN               1.0   

      ponds_nearest  days_exposition  
1097            NaN            147.0  
2033            NaN             34.0  
2603          374.0            276.0  
2632          374.0            256.0  
3574          603.0             45.0  
4151          722.0            869.0  
4189          953.0            223.0  
4670          376.0            350.0  
5343            NaN            303.0  
5707          170.0             49.0

In [24]:
display(
    real_estate_data.columns[real_estate_data.isna().any()].tolist()
)

['ceiling_height',
 'floors_total',
 'living_area',
 'kitchen_area',
 'locality_name',
 'airports_nearest',
 'city_centers_nearest',
 'parks_around3000',
 'parks_nearest',
 'ponds_around3000',
 'ponds_nearest',
 'days_exposition']

    ❌ 'ceiling_height' — сделаем `NaN` медианным значением по району
    ❌ 'floors_total' — сделаем `NaN` медианным значением по району
    ❌ 'living_area' — сделаем `NaN` медианным значением по диапазонам цен и рай
    ✅ 'is_apartment'
    ❌ 'kitchen_area' — если студия то кухня `NaN` проверим, если да то сделаем 0
    ✅ 'balcony'
    ❌ 'locality_name' — сначала удалим неявные дубликаты
    ❌ 'airports_nearest' — проверим по районам, может и нет ближайших аэропортов тогда `NaN` = 0
    ❌ 'cityCenters_nearest' — проверим по районам, может и нет ближайших центров города тогда `NaN` = 0
    ❌ 'parks_around3000' — проверим по районам, может и нет ближайших парков тогда `NaN` = 0
    ❌ 'parks_nearest'
    ❌ 'ponds_around3000' — проверим по районам, может и нет ближайших прудов тогда `NaN` = 0
    ❌ 'ponds_nearest'
    ❌ 'days_exposition' — посмотрим как связан столбец с `first_day_exposition`

**Предполагаю**

что стоит заменить пропущенные высоту потолков и количество этажей на медиану районных значений.

### `'locality_name'` — сначала удалим неявные дубликаты

func to reduce dublicates

In [25]:
# print(correct_name('поселок Мурино'))

In [26]:
real_estate_data['locality_name'].fillna('Санкт-Петербург', inplace=True)

In [27]:
# print(real_estate_data['locality_name'].mode())


print(len(real_estate_data['locality_name'].unique()))
real_estate_data['locality_name'] = real_estate_data['locality_name'].apply(correct_name)

# display(
#     real_estate_data['locality_name'].unique()[:5]
# )
print(len(real_estate_data['locality_name'].unique()))

# display(
#     list(real_estate_data['locality_name'].unique())
# )

364
330


type of location:
'деревня'
'садовое товарищество'
'городской посёлок'
'посёлок городского типа'
'коттеджный посёлок'
'посёлок при железнодорожной станции'
'посёлок станции'
'посёлок'
'село'
'садоводческое некоммерческое товарищество'

In [28]:
real_estate_data['locality_name'].describe()

count               23699
unique                330
top       Санкт-Петербург
freq                15770
Name: locality_name, dtype: object

In [29]:
real_estate_data['locality_name'], real_estate_data['locality_type'] = (
    zip(*real_estate_data['locality_name'].apply(name_n_type))
)

In [30]:
display(
    real_estate_data.head()
)

total_images  last_price  total_area first_day_exposition  rooms  \
0            20  13000000.0       108.0  2019-03-07T00:00:00      3   
1             7   3350000.0        40.4  2018-12-04T00:00:00      1   
2            10   5196000.0        56.0  2015-08-20T00:00:00      2   
3             0  64900000.0       159.0  2015-07-24T00:00:00      3   
4             2  10000000.0       100.0  2018-06-19T00:00:00      2   

   ceiling_height  floors_total  living_area  floor  is_apartment  studio  \
0            2.70          16.0         51.0      8         False   False   
1             NaN          11.0         18.6      1         False   False   
2             NaN           5.0         34.3      4         False   False   
3             NaN          14.0          NaN      9         False   False   
4            3.03          14.0         32.0     13         False   False   

   open_plan  kitchen_area  balcony    locality_name  airports_nearest  \
0      False          25.0    108.0  Санкт-Петербург           18863.0   
1      False          11.0      2.0           Шушары           12817.0   
2      False           8.3      0.0  Санкт-Петербург           21741.0   
3      False           NaN      0.0  Санкт-Петербург           28098.0   
4      False          41.0    108.0  Санкт-Петербург           31856.0   

   city_centers_nearest  parks_around3000  parks_nearest  ponds_around3000  \
0               16028.0               1.0          482.0               2.0   
1               18603.0               0.0            NaN               0.0   
2               13933.0               1.0           90.0               2.0   
3                6800.0               2.0           84.0               3.0   
4                8098.0               2.0          112.0               1.0   

   ponds_nearest  days_exposition locality_type  
0          755.0              NaN         город  
1            NaN             81.0       посёлок  
2          574.0            558.0         город  
3          234.0            424.0         город  
4           48.0            121.0         город

In [31]:
display(
    real_estate_data['locality_type'].unique()
)

array(['город', 'посёлок', 'городской посёлок', 'деревня',
       'посёлок городского типа', 'садовое товарищество', 'село',
       'посёлок городского типа имени', 'посёлок станции',
       'садоводческое некоммерческое товарищество',
       'посёлок 69-й километр', 'посёлок при железнодорожной станции',
       'коттеджный посёлок'], dtype=object)

In [32]:
real_estate_data.loc[real_estate_data['locality_name'] == 'посёлок 69-й километр', 'locality_name'] = '69-й километр'
real_estate_data.loc[real_estate_data['locality_type'] == 'посёлок 69-й километр', 'locality_type'] = 'посёлок'

In [33]:
display(
    real_estate_data['locality_type'].unique()
)

array(['город', 'посёлок', 'городской посёлок', 'деревня',
       'посёлок городского типа', 'садовое товарищество', 'село',
       'посёлок городского типа имени', 'посёлок станции',
       'садоводческое некоммерческое товарищество',
       'посёлок при железнодорожной станции', 'коттеджный посёлок'],
      dtype=object)

In [34]:
locality_type_series = pd.Series(real_estate_data['locality_type'].unique())
display(
    locality_type_series
)

0                                         город
1                                       посёлок
2                             городской посёлок
3                                       деревня
4                       посёлок городского типа
5                          садовое товарищество
6                                          село
7                 посёлок городского типа имени
8                               посёлок станции
9     садоводческое некоммерческое товарищество
10          посёлок при железнодорожной станции
11                           коттеджный посёлок
dtype: object

In [35]:
real_estate_data['locality_type'] = (
    real_estate_data['locality_type'].apply(lambda x: 
        list(locality_type_series[locality_type_series == x].index)[0]
    )
)

In [36]:
display(
    real_estate_data.head()
)

total_images  last_price  total_area first_day_exposition  rooms  \
0            20  13000000.0       108.0  2019-03-07T00:00:00      3   
1             7   3350000.0        40.4  2018-12-04T00:00:00      1   
2            10   5196000.0        56.0  2015-08-20T00:00:00      2   
3             0  64900000.0       159.0  2015-07-24T00:00:00      3   
4             2  10000000.0       100.0  2018-06-19T00:00:00      2   

   ceiling_height  floors_total  living_area  floor  is_apartment  studio  \
0            2.70          16.0         51.0      8         False   False   
1             NaN          11.0         18.6      1         False   False   
2             NaN           5.0         34.3      4         False   False   
3             NaN          14.0          NaN      9         False   False   
4            3.03          14.0         32.0     13         False   False   

   open_plan  kitchen_area  balcony    locality_name  airports_nearest  \
0      False          25.0    108.0  Санкт-Петербург           18863.0   
1      False          11.0      2.0           Шушары           12817.0   
2      False           8.3      0.0  Санкт-Петербург           21741.0   
3      False           NaN      0.0  Санкт-Петербург           28098.0   
4      False          41.0    108.0  Санкт-Петербург           31856.0   

   city_centers_nearest  parks_around3000  parks_nearest  ponds_around3000  \
0               16028.0               1.0          482.0               2.0   
1               18603.0               0.0            NaN               0.0   
2               13933.0               1.0           90.0               2.0   
3                6800.0               2.0           84.0               3.0   
4                8098.0               2.0          112.0               1.0   

   ponds_nearest  days_exposition  locality_type  
0          755.0              NaN              0  
1            NaN             81.0              1  
2          574.0            558.0              0  
3          234.0            424.0              0  
4           48.0            121.0              0

In [37]:
display(
    real_estate_data['locality_type'].unique()
)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [38]:
real_estate_data = real_estate_data.astype(
    {
        'locality_name': 'category',
        'locality_type': 'category',
        # 'balcony': 'uint8',
    }
)

why there is no inplace parameter in astype

In [39]:
display(
    real_estate_data['balcony'].unique()
)

array([108.,   2.,   0.,   1.,   5.,   4.,   3.])

In [40]:
display(
    real_estate_data.groupby(by=['locality_type'])['balcony'].median()
)
# ['balcony'].median()

locality_type
0       5.0
1       2.0
2     108.0
3       2.0
4       4.5
5      54.5
6     108.0
7       2.0
8     108.0
9     108.0
10    108.0
11      1.0
Name: balcony, dtype: float64

In [41]:
display(
    real_estate_data.columns
)

Index(['total_images', 'last_price', 'total_area', 'first_day_exposition',
       'rooms', 'ceiling_height', 'floors_total', 'living_area', 'floor',
       'is_apartment', 'studio', 'open_plan', 'kitchen_area', 'balcony',
       'locality_name', 'airports_nearest', 'city_centers_nearest',
       'parks_around3000', 'parks_nearest', 'ponds_around3000',
       'ponds_nearest', 'days_exposition', 'locality_type'],
      dtype='object')

In [42]:
types_na_balcony = (
    list(real_estate_data['locality_type']
             .loc[real_estate_data['balcony'] == 108]
             .unique())
)

for type_loc in types_na_balcony:
    
    balcony_mode_by_type = (
        real_estate_data
            .loc[(real_estate_data['locality_type'] == type_loc)
                & ~(real_estate_data['balcony'] == 108), ['balcony']]
            .mode()
    )
    
    if len(balcony_mode_by_type.index) == 0:
        balcony_mode_by_type = 0
    elif len(balcony_mode_by_type.index) == 1:
        balcony_mode_by_type = balcony_mode_by_type.iloc[0].item()
    else:
        balcony_mode_by_type = balcony_mode_by_type.sample().iloc[0].item()
    
    real_estate_data.loc[(real_estate_data['locality_type'] == type_loc)
                & (real_estate_data['balcony'] == 108), 'balcony'] = (
        real_estate_data
            .loc[(real_estate_data['locality_type'] == type_loc)
                & (real_estate_data['balcony'] == 108), 'balcony']
            .apply(lambda x: balcony_mode_by_type if bool(random.getrandbits(1)) else 0)
    )


In [43]:
display(
    real_estate_data['balcony'].unique()
)

array([1., 2., 0., 5., 4., 3.])

In [44]:
real_estate_data = real_estate_data.astype(
    {
        'balcony': 'uint8',
    }
)

In [45]:
display(
    real_estate_data['balcony'].info()
)

<class 'pandas.core.series.Series'>
RangeIndex: 23699 entries, 0 to 23698
Series name: balcony
Non-Null Count  Dtype
--------------  -----
23699 non-null  uint8
dtypes: uint8(1)
memory usage: 23.3 KB


None

    ❌ 'ceiling_height' — сделаем `NaN` медианным значением по району

In [46]:
types_na_ceiling = (
    list(real_estate_data['locality_type']
             .loc[real_estate_data['ceiling_height'].isna()]
             .unique())
)

for type_ceiling in types_na_ceiling:
    
    ceiling_median_by_type = (
        real_estate_data
            .loc[(real_estate_data['locality_type'] == type_ceiling)
                & ~(real_estate_data['ceiling_height'].isna()), 'ceiling_height']
            .median()
    )
    
    if math.isnan(ceiling_median_by_type):
        ceiling_median_by_type = real_estate_data['ceiling_height'].median()

    real_estate_data.loc[
        real_estate_data['locality_type'] == type_ceiling, 
        'ceiling_height'
    ] = real_estate_data.loc[
            real_estate_data['locality_type'] == type_ceiling, 
            'ceiling_height'
        ].fillna(value=ceiling_median_by_type)

# somehow .fillna(inplace=True) didn't work

In [47]:
real_estate_data['ceiling_height'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23699 entries, 0 to 23698
Series name: ceiling_height
Non-Null Count  Dtype  
--------------  -----  
23699 non-null  float64
dtypes: float64(1)
memory usage: 185.3 KB


In [48]:
# display(
#     sorted(real_estate_data['ceiling_height'].unique())
# )

In [49]:
display(
    real_estate_data['ceiling_height'].describe()
)

count    23699.000000
mean         2.721105
std          0.988643
min          1.000000
25%          2.600000
50%          2.650000
75%          2.700000
max        100.000000
Name: ceiling_height, dtype: float64

In [50]:
types_odd_ceiling = (
    list(real_estate_data.loc[
            (real_estate_data['ceiling_height'] <= 2)
            | (real_estate_data['ceiling_height'] > 60)
            | ((real_estate_data['ceiling_height'] > 6)
                & (real_estate_data['ceiling_height'] < 20)
              ), 'locality_type'
        ].unique()
    )
)


for type_ceiling in types_odd_ceiling:
    
    ceiling_median_by_type = (
        real_estate_data.loc[
            (real_estate_data['locality_type'] == type_ceiling)
                & (real_estate_data['ceiling_height'] <= 6)
                & (real_estate_data['ceiling_height'] > 2)
            , 'ceiling_height'
        ].median()
    )

    display(ceiling_median_by_type)
    
    
    if math.isnan(ceiling_median_by_type):
        ceiling_median_by_type = real_estate_data['ceiling_height'].median()

    real_estate_data.loc[
            (real_estate_data['locality_type'] == type_ceiling)
            & (real_estate_data['ceiling_height'] <= 2)
            | (real_estate_data['ceiling_height'] > 60)
            | ((real_estate_data['ceiling_height'] > 6)
                & (real_estate_data['ceiling_height'] < 20)
              )
            , 'ceiling_height'
    ] = ceiling_median_by_type

2.65

2.65

2.56

2.6

In [51]:
display(
    real_estate_data['ceiling_height'].describe()
)

count    23699.000000
mean         2.715745
std          0.751619
min          2.200000
25%          2.600000
50%          2.650000
75%          2.700000
max         32.000000
Name: ceiling_height, dtype: float64

In [52]:
real_estate_data.loc[
            (real_estate_data['ceiling_height'] >= 20)
            & (real_estate_data['ceiling_height'] <= 60)
            , 'ceiling_height'
] = real_estate_data.loc[
            (real_estate_data['ceiling_height'] >= 20)
            & (real_estate_data['ceiling_height'] <= 60)
            , 'ceiling_height'
    ].apply(lambda x: x / 10)

In [53]:
display(
    real_estate_data['ceiling_height'].describe()
)

count    23699.000000
mean         2.693905
std          0.221764
min          2.000000
25%          2.600000
50%          2.650000
75%          2.700000
max          6.000000
Name: ceiling_height, dtype: float64

    ❌ 'floors_total' — сделаем `NaN` медианным значением по району

In [54]:
display(
    real_estate_data.loc[
        real_estate_data['floors_total'].isna()
    ].groupby(by='floor')['floor'].count()
)

floor
1      3
2      2
3      4
4     10
5      4
6      9
7      4
8      8
9      5
10     3
11     3
12     6
13     3
14     2
15     3
17     1
18     3
19     3
20     2
21     4
22     1
23     1
24     2
Name: floor, dtype: int64

seems missing `'floors_total'` not depends on `'floor'`

In [55]:
display(
    real_estate_data.loc[real_estate_data['floor'] > real_estate_data['floors_total']]
)

Empty DataFrame
Columns: [total_images, last_price, total_area, first_day_exposition, rooms, ceiling_height, floors_total, living_area, floor, is_apartment, studio, open_plan, kitchen_area, balcony, locality_name, airports_nearest, city_centers_nearest, parks_around3000, parks_nearest, ponds_around3000, ponds_nearest, days_exposition, locality_type]
Index: []

In [56]:
display(
    real_estate_data.loc[
        real_estate_data['floors_total'].isna()
    ].sample(n=10)
)

total_images  last_price  total_area first_day_exposition  rooms  \
237               4   2438033.0       28.10  2016-11-23T00:00:00      1   
11328            10   5386325.0       58.13  2018-10-10T00:00:00      3   
5958              1   3200000.0       30.30  2018-09-07T00:00:00      0   
7662              4   8220312.0       80.12  2017-05-06T00:00:00      2   
10486             5   6300000.0       68.20  2018-04-01T00:00:00      2   
186              12  11640000.0       65.20  2018-10-02T00:00:00      2   
10983             6   4670000.0       37.80  2019-03-13T00:00:00      1   
13438             4   4403541.0       65.30  2016-09-21T00:00:00      2   
17925            10   3241540.0       32.69  2018-10-01T00:00:00      1   
13622             3   7650000.0       88.84  2016-05-21T00:00:00      3   

       ceiling_height  floors_total  living_area  floor  is_apartment  studio  \
237              2.65           NaN        20.75      1         False   False   
11328            2.65           NaN        31.00      9         False   False   
5958             2.65           NaN          NaN     13         False    True   
7662             2.65           NaN          NaN      6         False   False   
10486            2.65           NaN          NaN      4         False   False   
186              2.65           NaN        30.80      4         False   False   
10983            2.65           NaN        16.90     14         False   False   
13438            2.65           NaN        31.60     11         False   False   
17925            2.65           NaN        11.00      9         False   False   
13622            2.65           NaN        49.87     21         False   False   

       open_plan  kitchen_area  balcony    locality_name  airports_nearest  \
237        False           NaN        0  Санкт-Петербург           22041.0   
11328      False           NaN        1  Санкт-Петербург           47303.0   
5958       False           NaN        0  Санкт-Петербург           22735.0   
7662       False           NaN        0  Санкт-Петербург           17720.0   
10486      False           NaN        0  Санкт-Петербург           43642.0   
186        False         12.00        0  Санкт-Петербург           39197.0   
10983      False          9.50        0  Санкт-Петербург           23982.0   
13438      False         12.20        0  Санкт-Петербург           22041.0   
17925      False           NaN        1  Санкт-Петербург           47303.0   
13622      False          9.76        0  Санкт-Петербург           35234.0   

       city_centers_nearest  parks_around3000  parks_nearest  \
237                 17369.0               0.0            NaN   
11328               25865.0               1.0          400.0   
5958                11618.0               1.0          835.0   
7662                16722.0               3.0          471.0   
10486                9496.0               0.0            NaN   
186                 12373.0               1.0          123.0   
10983               11634.0               0.0            NaN   
13438               17369.0               0.0            NaN   
17925               25866.0               1.0          251.0   
13622               14674.0               0.0            NaN   

       ponds_around3000  ponds_nearest  days_exposition locality_type  
237                 1.0          374.0            251.0             0  
11328               1.0          492.0            135.0             0  
5958                1.0          652.0             68.0             0  
7662                2.0          440.0             93.0             0  
10486               0.0            NaN            179.0             0  
186                 0.0            NaN             49.0             0  
10983               0.0            NaN              NaN             0  
13438               1.0          374.0            320.0             0  
17925               1.0          350.0            145.0             0  
13622

In [57]:
types_na_flrs_ttl = (
    list(real_estate_data.loc[
            (real_estate_data['floors_total'].isna())
            , 'locality_type'
        ].unique()
    )
)

print(types_na_flrs_ttl)

for type_flrs_ttl in types_na_flrs_ttl:
    
    flrs_ttl_median_by_type = (
        real_estate_data.loc[
            (real_estate_data['locality_type'] == type_flrs_ttl)
            & ~(real_estate_data['floors_total'].isna())
            , 'floors_total'
        ].median()
    )

    print(flrs_ttl_median_by_type)
    
    
    if math.isnan(flrs_ttl_median_by_type):
        flrs_ttl_median_by_type = real_estate_data['floors_total'].median()
        print('NANы', flrs_ttl_median_by_type)
        
#     real_estate_data.loc[
#         (real_estate_data['locality_type'] == type_flrs_ttl)
#         & (real_estate_data['floors_total'].isna())
#         , 'floors_total'
#     ].fillna(value=flrs_ttl_median_by_type, inplace=True)
    
    real_estate_data.loc[
            (real_estate_data['locality_type'] == type_flrs_ttl)
            & (real_estate_data['floors_total'].isna())
            , 'floors_total'
    ] = flrs_ttl_median_by_type
    
    # real_estate_data.loc[
    #     (real_estate_data['locality_type'] == type_flrs_ttl)
    #     & (real_estate_data['floors_total'].isna())
    #     , 'floors_total'
    # ] = real_estate_data.loc[
    #         (real_estate_data['locality_type'] == type_flrs_ttl)
    #         & (real_estate_data['floors_total'].isna())
    #         , 'floors_total'
    #     ].fillna(value=flrs_ttl_median_by_type)

[0, 1, 2, 4, 3]
9.0
12.0
5.0
5.0
9.0


🤦‍♂️

In [58]:
#     why ↓ it's not working in for loop!?
    
    # real_estate_data.loc[
    #     (real_estate_data['locality_type'] == type_flrs_ttl)
    #     & (real_estate_data['floors_total'].isna())
    #     , 'floors_total'
    # ].fillna(value=flrs_ttl_median_by_type, inplace=True)

In [59]:
display(
    # real_estate_data.loc[
    #     real_estate_data['floors_total'].isna()
    # ].sample(n=10)
    
        real_estate_data['floors_total'].describe(),
        real_estate_data['floors_total'].info()
)

<class 'pandas.core.series.Series'>
RangeIndex: 23699 entries, 0 to 23698
Series name: floors_total
Non-Null Count  Dtype  
--------------  -----  
23699 non-null  float64
dtypes: float64(1)
memory usage: 185.3 KB


count    23699.000000
mean        10.668425
std          6.586123
min          1.000000
25%          5.000000
50%          9.000000
75%         16.000000
max         60.000000
Name: floors_total, dtype: float64

None

In [60]:
display(
    real_estate_data.loc[real_estate_data['floor'] > real_estate_data['floors_total'], 'floor'].describe()
)

count    36.000000
mean     16.166667
std       4.358899
min      10.000000
25%      12.000000
50%      15.000000
75%      20.000000
max      24.000000
Name: floor, dtype: float64

In [61]:
real_estate_data.loc[
    (real_estate_data['floor'] > real_estate_data['floors_total'])
    , 'floor'
] = real_estate_data.loc[
        (real_estate_data['floor'] > real_estate_data['floors_total'])
        , 'floors_total'
    ]

In [62]:
display(
    real_estate_data.loc[real_estate_data['floor'] > real_estate_data['floors_total'], 'floor'].count()
)

0

In [63]:
real_estate_data = real_estate_data.astype(
    {
        'floors_total': 'uint8',
    }
)

    ❌ 'living_area' — сделаем `NaN` медианным значением по диапазонам цен и району

In [64]:
# display(
#     real_estate_data.loc[
#         real_estate_data['living_area'].isna()
#         , 'floor'
#     ].count()
# )

In [65]:
display(
    real_estate_data['total_area'].describe()
)

count    23699.000000
mean        60.348651
std         35.654083
min         12.000000
25%         40.000000
50%         52.000000
75%         69.900000
max        900.000000
Name: total_area, dtype: float64

In [66]:
display(
    real_estate_data.loc[
        real_estate_data['total_area'] > 450
    ]
)

total_images   last_price  total_area first_day_exposition  rooms  \
660               3   49950000.0       483.9  2017-10-04T00:00:00      7   
3117             19  140000000.0       631.0  2019-03-30T00:00:00      7   
4237              1   50000000.0       517.0  2017-10-14T00:00:00      7   
5358             20   65000000.0       590.0  2019-02-14T00:00:00     15   
5893              3  230000000.0       500.0  2017-05-31T00:00:00      6   
6221              8   34000000.0       470.3  2018-01-14T00:00:00      4   
6621             20   99000000.0       488.0  2017-04-09T00:00:00      5   
8018             20   84000000.0       507.0  2015-12-17T00:00:00      5   
9826              1   43185328.0       494.1  2017-05-12T00:00:00      1   
12401            20   91500000.0       495.0  2017-06-19T00:00:00      7   
12859            20  140000000.0       631.2  2019-04-09T00:00:00      7   
15016             8  150000000.0       500.0  2016-01-13T00:00:00      4   
15651            20  300000000.0       618.0  2017-12-18T00:00:00      7   
19540             8  420000000.0       900.0  2017-12-06T00:00:00     12   
20273            12   20000000.0       460.8  2019-03-20T00:00:00      6   
22494             7   91075000.0       491.0  2017-05-27T00:00:00      5   

       ceiling_height  floors_total  living_area  floor  is_apartment  studio  \
660              3.20             6        312.5      4         False   False   
3117             2.65             5          NaN      5         False   False   
4237             2.65             4        332.0      3         False   False   
5358             3.50             6        409.0      3         False   False   
5893             2.65             7          NaN      7         False   False   
6221             2.65             3        248.7      3         False   False   
6621             2.95            20        216.0     17         False   False   
8018             4.45             7        301.5      7          True   False   
9826             2.85            12          NaN      3         False   False   
12401            4.65             7        347.5      7         False   False   
12859            3.90             4        322.3      4         False   False   
15016            3.20             7          NaN      7         False   False   
15651            3.40             7        258.0      5         False   False   
19540            2.80            25        409.7     25         False   False   
20273            3.10             3        279.6      1         False   False   
22494            4.20             9        274.0      9         False   False   

       open_plan  kitchen_area  balcony    locality_name  airports_nearest  \
660        False          26.4        1  Санкт-Петербург           23079.0   
3117       False          60.0        1  Санкт-Петербург           25707.0   
4237       False          22.0        1  Санкт-Петербург           22835.0   
5358       False         100.0        1  Санкт-Петербург           24447.0   
5893       False          40.0        0  Санкт-Петербург           32440.0   
6221       False          40.0        0  Санкт-Петербург           43756.0   
6621       False          50.0        1  Санкт-Петербург           27641.0   
8018       False          45.5        1  Санкт-Петербург               NaN   
9826       False           NaN        0  Санкт-Петербург           28529.0   
12401      False          25.0        0  Санкт-Петербург               NaN   
12859      False          19.5        1  Санкт-Петербург           25707.0   
15016      False           NaN        0  Санкт-Петербург           33058.0   
15651      False          70.0        1  Санкт-Петербург           32440.0   
19540      False         112.0        1  Санкт-Петербург           30706.0   
20273      False          55.5        0  Санкт-Петербург           43756.0   
22494      False          45.0        0  Санкт-Петербург           25525.0   

       ci

In [67]:
display(
    real_estate_data['living_area'].describe()
)

count    21796.000000
mean        34.457852
std         22.030445
min          2.000000
25%         18.600000
50%         30.000000
75%         42.300000
max        409.700000
Name: living_area, dtype: float64

In [68]:
types_na_living_area = (
    list(real_estate_data.loc[
            (real_estate_data['living_area'].isna())
            , 'locality_type'
        ].unique()
    )
)


for type_living_area in types_na_living_area:
    
    living_area_median_by_type = (
        real_estate_data.loc[
            (real_estate_data['locality_type'] == type_living_area)
            & ~(real_estate_data['living_area'].isna())
            , 'living_area'
        ].median()
    )
    
    living_part_median_by_type = round(pd.Series(
        real_estate_data.loc[
            (real_estate_data['locality_type'] == type_living_area)
            & (real_estate_data['living_area'] > 8)
            & ~(real_estate_data['living_area'].isna())
            , 'living_area'
        ] / real_estate_data.loc[
            (real_estate_data['locality_type'] == type_living_area)
            & (real_estate_data['living_area'] > 8)
            & ~(real_estate_data['living_area'].isna())
            , 'total_area'
        ]
    ).median(), 2)

    if math.isnan(living_area_median_by_type):
        living_area_median_by_type = real_estate_data['living_area'].median()
        print('NANы')
        
    real_estate_data.loc[
            (real_estate_data['locality_type'] == type_living_area)
            & ((real_estate_data['living_area'].isna())
            | (real_estate_data['living_area'] < 8))
            , 'living_area'
    ] = real_estate_data.loc[
            (real_estate_data['locality_type'] == type_living_area)
            & ((real_estate_data['living_area'].isna())
            | (real_estate_data['living_area'] < 8))
            , 'total_area'
        ].apply(lambda x: x * living_part_median_by_type)

In [69]:
display(
    real_estate_data['living_area'].describe()
)

count    23699.000000
mean        34.579383
std         22.347244
min          7.410000
25%         19.000000
50%         30.000000
75%         42.100000
max        409.700000
Name: living_area, dtype: float64

❌ 'kitchen_area' — если студия то кухня `NaN` проверим, если да то сделаем 0

In [70]:
display(
    real_estate_data.loc[real_estate_data['kitchen_area'].isna()
                         , 'living_area'].count()
)

2278

In [71]:
display(
    real_estate_data.loc[
        (real_estate_data['kitchen_area'].isna())
        & (real_estate_data['studio'])
        , 'living_area'
    ].count()
)

149

In [72]:
display(
    real_estate_data.loc[
        ~ (real_estate_data['kitchen_area'].isna())
        & (real_estate_data['studio'])
        , 'living_area'
    ].count()
)

0

😏

In [73]:
display(
    real_estate_data.loc[
        (real_estate_data['kitchen_area'].isna())
        & (real_estate_data['is_apartment'])
        , 'living_area'
    ].count()
)

15

In [74]:
display(
    real_estate_data.loc[
        ~ (real_estate_data['kitchen_area'].isna())
        & (real_estate_data['is_apartment'])
    ].sample(10)
)

total_images  last_price  total_area first_day_exposition  rooms  \
8611              6   6030000.0        50.4  2018-02-20T00:00:00      2   
2975              9   4931398.0        42.9  2018-07-23T00:00:00      1   
21458             7   5616379.0        47.4  2018-07-23T00:00:00      1   
16166            13   4792000.0        49.0  2017-08-07T00:00:00      1   
3951             10   4300000.0        50.0  2019-04-20T00:00:00      1   
19017            12   4887000.0        52.4  2018-03-19T00:00:00      1   
8252             15  12750000.0       102.0  2018-10-17T00:00:00      3   
20008             8   5400000.0        31.0  2018-02-21T00:00:00      1   
23236            18  13900000.0        95.0  2017-10-15T00:00:00      3   
3067             13   6600000.0        95.0  2018-09-08T00:00:00      2   

       ceiling_height  floors_total  living_area  floor  is_apartment  studio  \
8611             3.10             4        37.00      2          True   False   
2975             3.15             4        20.39      3          True   False   
21458            3.00             4        20.52      3          True   False   
16166            3.12             4        21.00      3          True   False   
3951             3.00            11        17.00      9          True   False   
19017            3.10             4        24.70      2          True   False   
8252             2.80             7        61.00      4          True   False   
20008            2.75             9        20.00      5          True   False   
23236            2.80            15        50.00     15          True   False   
3067             4.50             7        45.00      7          True   False   

       open_plan  kitchen_area  balcony    locality_name  airports_nearest  \
8611       False         11.00        0  Санкт-Петербург           45767.0   
2975       False         10.29        0  Санкт-Петербург           44870.0   
21458      False         14.04        2  Санкт-Петербург           44870.0   
16166      False         13.60        2  Санкт-Петербург           44870.0   
3951       False         12.00        2  Санкт-Петербург           10143.0   
19017      False         11.70        5  Санкт-Петербург           44870.0   
8252       False         12.00        5          Колпино           26100.0   
20008      False          6.50        0  Санкт-Петербург           27747.0   
23236      False         11.00        4  Санкт-Петербург           27573.0   
3067       False         25.00        2           Выборг               NaN   

       city_centers_nearest  parks_around3000  parks_nearest  \
8611                21700.0               0.0            NaN   
2975                20802.0               0.0            NaN   
21458               20802.0               0.0            NaN   
16166               20802.0               0.0            NaN   
3951                11367.0               0.0            NaN   
19017               20802.0               0.0            NaN   
8252                31886.0               2.0          332.0   
20008                6450.0               3.0          246.0   
23236                9436.0               1.0          714.0   
3067                    NaN               NaN            NaN   

       ponds_around3000  ponds_nearest  days_exposition locality_type  
8611                1.0          576.0              NaN             0  
2975                1.0          792.0            273.0             0  
21458               0.0            NaN              NaN             0  
16166               1.0          792.0            337.0             0  
3951                0.0            NaN              NaN             0  
19017               1.0          792.0            113.0             0  
8252                2.0          472.0            172.0             0  
20008               3.0          177.0              8.0             0  
23236               0.0            NaN              7.0             0  
3067 

In [75]:
real_estate_data.loc[
        (real_estate_data['kitchen_area'].isna())
        & ((real_estate_data['studio'])
          | (real_estate_data['is_apartment']))
        , 'kitchen_area'
] = 0

In [76]:
types_na_kitchen_area = (
    list(real_estate_data.loc[
            (real_estate_data['kitchen_area'].isna())
            , 'locality_type'
        ].unique()
    )
)

for type_kitchen_area in types_na_kitchen_area:
    
    kitchen_area_part_mdn_by_type = round(pd.Series(
        real_estate_data.loc[
            (real_estate_data['locality_type'] == type_kitchen_area)
            & ~(real_estate_data['kitchen_area'].isna())
            , 'kitchen_area'
        ] / real_estate_data.loc[
            (real_estate_data['locality_type'] == type_kitchen_area)
            & ~(real_estate_data['kitchen_area'].isna())
            , 'total_area'
        ]
    ).median(), 2)
    
    if math.isnan(kitchen_area_part_mdn_by_type):
        print('NANы kitchen_area_part_mdn_by_type', type_kitchen_area)
        
    real_estate_data.loc[
            (real_estate_data['locality_type'] == type_kitchen_area)
            & (real_estate_data['kitchen_area'].isna())
            , 'kitchen_area'
    ] = real_estate_data.loc[
            (real_estate_data['locality_type'] == type_kitchen_area)
            & (real_estate_data['kitchen_area'].isna())
            , 'total_area'
        ].apply(lambda x: x * kitchen_area_part_mdn_by_type)


In [77]:
display(
    real_estate_data['kitchen_area'].describe()
)

count    23699.000000
mean        10.460818
std          6.062755
min          0.000000
25%          7.000000
50%          9.000000
75%         12.000000
max        112.000000
Name: kitchen_area, dtype: float64

❌ 'airports_nearest' — проверим по районам, может и нет ближайших аэропортов тогда `NaN` = 0

In [78]:
display(
    real_estate_data['airports_nearest'].describe()
)

count    18157.000000
mean     28793.672193
std      12630.880622
min          0.000000
25%      18585.000000
50%      26726.000000
75%      37273.000000
max      84869.000000
Name: airports_nearest, dtype: float64

In [79]:
display(
    real_estate_data['airports_nearest'].info()
)

<class 'pandas.core.series.Series'>
RangeIndex: 23699 entries, 0 to 23698
Series name: airports_nearest
Non-Null Count  Dtype  
--------------  -----  
18157 non-null  float64
dtypes: float64(1)
memory usage: 185.3 KB


None

In [80]:
display(
    real_estate_data.loc[
        real_estate_data['airports_nearest'].isna()
        , 'locality_type'
    ].unique()
)

[2, 1, 0, 3, 4, ..., 7, 8, 9, 10, 11]
Length: 12
Categories (12, int64): [0, 1, 2, 3, ..., 8, 9, 10, 11]

In [81]:
real_estate_data.loc[
    real_estate_data['airports_nearest'] <= 1000
] = np.nan


In [82]:
display(np.nan)

nan

In [85]:
names_na_airprts_nrst = (
    list(real_estate_data.loc[
            (real_estate_data['airports_nearest'].isna())
            , 'locality_name'
        ].unique()
    )
)

for name_airprts_nrst in names_na_airprts_nrst:
    
    airprts_nrst_median_by_names = (
        real_estate_data.loc[
            (real_estate_data['locality_name'] == name_airprts_nrst)
            & ~(real_estate_data['airports_nearest'].isna())
            , 'airports_nearest'
        ].median()
    )

    if math.isnan(airprts_nrst_median_by_names):
        
        type_airprts_nrst = real_estate_data.loc[
                (real_estate_data['locality_name'] == name_airprts_nrst)
                , 'locality_type'
            ].iloc[0].item()
        airprts_nrst_median_by_names = real_estate_data.loc[
                (real_estate_data['locality_type'] == type_airprts_nrst)
                , 'airports_nearest'
            ]
        if not airprts_nrst_median_by_names.isna().all():
            airprts_nrst_median_by_names = airprts_nrst_median_by_names.median(skipna=True)
        else:  
            airprts_nrst_median_by_names = np.nan
        
    real_estate_data.loc[
            (real_estate_data['locality_name'] == name_airprts_nrst)
            & (real_estate_data['airports_nearest'].isna())
            , 'airports_nearest'
    ] = airprts_nrst_median_by_names

IndexError: single positional indexer is out-of-bounds

In [84]:
display(
    real_estate_data['airports_nearest'].describe()
)

count    18157.000000
mean     28793.672193
std      12630.880622
min          0.000000
25%      18585.000000
50%      26726.000000
75%      37273.000000
max      84869.000000
Name: airports_nearest, dtype: float64

In [ ]:
display(
    real_estate_data.loc[real_estate_data['locality_name'].isna()]['is_apartment'].unique()
)

In [ ]:
display(
    real_estate_data.loc[real_estate_data['locality_name'].isna()]['is_apartment'].count()
)

Так, похоже, что у всех `'locality_name'` = `NaN`, `'is_apartment'` = `False`

In [ ]:
display(
    len(real_estate_data.loc[real_estate_data['parks_around3000'].isna()]['parks_nearest']),
    len(real_estate_data.loc[real_estate_data['parks_around3000'] == 0]['parks_nearest'].unique()),
    (real_estate_data.loc[(real_estate_data['parks_around3000'] == 0)
                             and (real_estate_data['parks_nearest'].isna())]['rooms'].count()),
    
)

In [ ]:
real_estate_data['first_day_exposition'] = pd.to_datetime(real_estate_data['first_day_exposition'], format='%d/%m/%YT%h%m%s')

In [ ]:
display(
    real_estate_data['first_day_exposition'].describe()
)

In [ ]:
display(
    real_estate_data.loc[real_estate_data['locality_name'].isna()].head()
)
#real_estate_data['locality_name'].fillna(value=False, inplace=True)


In [ ]:
display(
    real_estate_data.loc[real_estate_data['cityCenters_nearest'].isna()]['locality_name'].unique(),
    len(real_estate_data.loc[real_estate_data['cityCenters_nearest'].isna()]['locality_name'].unique())
)
#real_estate_data['locality_name'].fillna(value=False, inplace=True)


In [ ]:
display(
    
    real_estate_data['cityCenters_nearest'] = (
        real_estate_data.loc[real_estate_data['cityCenters_nearest'].isna()]['locality_name'].unique()
    ),
    len(real_estate_data.loc[real_estate_data['cityCenters_nearest'].isna()]['locality_name'].unique())
)
#real_estate_data['locality_name'].fillna(value=False, inplace=True)


In [ ]:
display(
    real_estate_data.loc[real_estate_data['studio'] == True].head(),
    len(real_estate_data.loc[real_estate_data['studio'] == True])
)
#real_estate_data['locality_name'].fillna(value=False, inplace=True)


`ceiling_height` — пропуски заменим медианой по району `locality_name`

In [ ]:
display(real_estate_data.info())

Приведите данные к нужным типам:
Поясните, в каких столбцах нужно изменить тип данных и почему.

In [ ]:
real_estate_data.astype(
        {
            'total_images': 'uint16',
            'rooms':'uint8',
            'ceiling_height':'float16',
            'floors_total':'uint16',
            'floor':'int16',
            'parks_around3000':'uint8',
            'ponds_around3000':'uint8',
            'is_apartment': 'bool',
        }
    ).dtypes


## Шаг 3. Посчитайте и добавьте в таблицу

цену квадратного метра;
день недели, месяц и год публикации объявления;
этаж квартиры; варианты — первый, последний, другой;
соотношение жилой и общей площади, а также отношение площади кухни к общей.

## Шаг 4. Проведите исследовательский анализ данных

Изучите следующие параметры: площадь, цена, число комнат, высота потолков. Постройте гистограммы для каждого параметра.
Изучите время продажи квартиры. Постройте гистограмму. Посчитайте среднее и медиану. Опишите, сколько обычно занимает продажа. Когда можно считать, что продажи прошли очень быстро, а когда необычно долго?
Уберите редкие и выбивающиеся значения. Опишите, какие особенности обнаружили.
Какие факторы больше всего влияют на стоимость квартиры? Изучите, зависит ли цена от площади, числа комнат, удалённости от центра. Изучите зависимость цены от того, на каком этаже расположена квартира: первом, последнем или другом. Также изучите зависимость от даты размещения: дня недели, месяца и года.
Выберите 10 населённых пунктов с наибольшим числом объявлений. Посчитайте среднюю цену квадратного метра в этих населённых пунктах. Выделите среди них населённые пункты с самой высокой и низкой стоимостью жилья. Эти данные можно найти по имени в столбце locality_name.
Изучите предложения квартир: для каждой квартиры есть информация о расстоянии до центра. Выделите квартиры в Санкт-Петербурге (locality_name). Ваша задача — выяснить, какая область входит в центр. Создайте столбец с расстоянием до центра в километрах: округлите до целых значений. После этого посчитайте среднюю цену для каждого километра. Постройте график: он должен показывать, как цена зависит от удалённости от центра. Определите границу, где график сильно меняется, — это и будет центральная зона.
Выделите сегмент квартир в центре. Проанализируйте эту территорию и изучите следующие параметры: площадь, цена, число комнат, высота потолков. Также выделите факторы, которые влияют на стоимость квартиры (число комнат, этаж, удалённость от центра, дата размещения объявления). Сделайте выводы. Отличаются ли они от общих выводов по всей базе?

## Шаг 5. Напишите общий вывод

## Описание данных

`airports_nearest` — расстояние до ближайшего аэропорта в метрах (м)

`balcony` — число балконов

`ceiling_height` — высота потолков (м)

`cityCenters_nearest` — расстояние до центра города (м)

`days_exposition` — сколько дней было размещено объявление (от публикации до снятия)

`first_day_exposition` — дата публикации

`floor` — этаж

`floors_total` — всего этажей в доме

`is_apartment` — апартаменты (булев тип)

`kitchen_area` — площадь кухни в квадратных метрах (м²)

`last_price` — цена на момент снятия с публикации

`living_area` — жилая площадь в квадратных метрах (м²)

`locality_name` — название населённого пункта

`open_plan` — свободная планировка (булев тип)

`parks_around3000` — число парков в радиусе 3 км

`parks_nearest` — расстояние до ближайшего парка (м)

`ponds_around3000` — число водоёмов в радиусе 3 км

`ponds_nearest` — расстояние до ближайшего водоёма (м)

`rooms` — число комнат

`studio` — квартира-студия (булев тип)

`total_area` — площадь квартиры в квадратных метрах (м²)

`total_images` — число фотографий квартиры в объявлении